In [14]:
import pandas as pd
import os
import re
import warnings
import shutil
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select
import time

warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")

In [11]:
def leitura_bases(dados_ssp_sp, dict_bases):
    """
    Carregar os arquivos baixados a partir do script 'web_scrapping_bases.py'.
    Esse serão organizados em um dicionário segmentado por ano, região e município.
    
    Args:
        dados_ssp_sp O nome da pasta criada no "web_scrapping_bases.py"
        dict_bases: Dicionário que armezanará os dados carregados

    Returns:
        O dicionário atualizado com os Dataframes processados
    """
    for ano in sorted(os.listdir(dados_ssp_sp), reverse=True):
        dict_bases[ano] = {}
        for regiao in os.listdir(f"{dados_ssp_sp}/{ano}"):
            dict_bases[ano][regiao] = {}
            for municipio in os.listdir(f"{dados_ssp_sp}/{ano}/{regiao}"):
                df = leitura_municipio(ano, regiao, municipio)
                dict_bases[ano][regiao][municipio] = df
    
    return dict_bases

def leitura_municipio(ano, regiao, municipio):
    """
    Função auxiliar para a leitura dos arquivos .xls
    Args:
        ano: Ano referente aos crimes ocorridos
        regiao: Região do estado de SP referente aos crimes ocorridos
        municipio: Município referente aos crimes ocorridos

    Returns:
        Dataframe Pandas do município para um dado ano
    """
    dir = os.path.join(os.getcwd(), "Dados_ssp_sp", ano, regiao, municipio)
    caminho = str(dir) + "\\" + os.listdir(dir)[0]
    df = pd.read_excel(caminho)
    return df
    

def formatar_nome(crime):
    """
    Função auxiliar para formatar os tipos de crime
    
    Args:
        o tipo do crime 

    Returns:
        O nome formatado 
    """
    crime_normalizado = re.sub(r'\W+', '_', crime)
    crime_normalizado = crime_normalizado.strip('_')
    crime_normalizado = crime_normalizado.lower()
    return crime_normalizado


def corrigir_valores_coluna(valor):
    """
    Função para corrigir formatações indevidas dos dados.
    Muitos valores foram registrados como float de forma errada: 
    
    Alguns exemplos,
    1.543 -> 1543
    1.45 -> 1450
    28.000 -> 28
    
    Além disso, para padronizar, transformou-se os valores "...", referentes
    a dados não registrados para 0.
    
    """    
    if isinstance(valor, str) and "..." in valor:
        return 0
    
    if isinstance(valor, (int, float)):
        valor_str = str(valor)
        
        if '.' in valor_str:
            parte_inteira, parte_decimal = valor_str.split('.')

            if parte_decimal != '0' * len(parte_decimal):
                return int(float(valor) * 1000)
            else:
                return int(parte_inteira)
    
        return int(valor)

    return valor
        
def processar_base_municipio(pre_df, nome_mun, codigos_mun, ano):
    """
    Para cada município, faz-se um dicionário no qual cada chave refere-se 
    a um crime e cada valor um dataframe que possui o número de ocorrencias
    no decorrer do ano em questão, segmentado por mês.

    Args:
        pre_df: o arquivo .xls gerado a partir do site SSP-SP, sem nenhuma alteração 
        nome_mun: o nome do município que será processado
        codigos_mun: Dataframe com os códigos e nomes dos municípios fornecido pela SEADE 
        ano: ano referente às ocorrencias criminais

    Returns:
        Dicionário no formato {'nome_do_crime': 'Ocorrencias_mensais_do_crime'}
        
    """
    df = pre_df.copy()

    # 1. Ajuste de tipos e renomeação das colunas
    for coluna in df.columns[1:]:
        df[coluna] = df[coluna].apply(corrigir_valores_coluna)    
    df.rename(columns=lambda x: f"{x} {ano}" if x in ['Janeiro', 'Fevereiro', 'Marco',
                                                       'Abril', 'Maio', 'Junho',
                                                       'Julho', 'Agosto','Setembro',
                                                       'Outubro', 'Novembro', 'Dezembro'] else x, inplace=True)
    
    # 2. Criação de um dataset para cada tipo de crime. 
    dicionario_df_crimes = {}
    for crime in df['Natureza'].unique():
        df_crime = df[df['Natureza'] == crime]

        dicionario_df_crimes[formatar_nome(crime)] = df_crime.drop(columns=['Natureza'])

    # 3. Associar as bases aos seus respectivos códigos e nomes do município
    for crime, df_crime in dicionario_df_crimes.items():
    # Merge dos dados criminais com os códigos para Junções Futuras
        df_crime['municipios'] = nome_mun
        df_crime = pd.merge(df_crime, codigos_mun, on='municipios', how='inner')

        # Reajuste da ordem das colunas
        cols = df_crime.columns.tolist()
        cols_em_primeiro = cols[-4:]
        nova_ordem_cols = cols_em_primeiro + [col for col in cols if col not in cols_em_primeiro]
        df_crime = df_crime[nova_ordem_cols]

        # Atribuindo os dataframes modificados ao dicionário
        dicionario_df_crimes[crime] = df_crime
    
    return dicionario_df_crimes
    

def concatenar_municipios(dict_novo_municipio, dict_consolidado=None):
    """
    Função para concatenar os dataframes dos crimes dos municípios.

    Args:
        dict_novo_municipio: Dicionário no formato do output da função 'processar_base_municipio()'
        dict_consolidado: Dicionário final que terá como chave o ano e como valor os dataframes
        referentes a cada crime, com todos os municípios processados, junto de seus códigos.

    Returns:
        Atualização do dict_consolidado, com o novo município concatenado
    """
    
    if dict_consolidado is None:
        dict_consolidado = {}

    for crime, df in dict_novo_municipio.items():
        if crime in dict_consolidado:
            dict_consolidado[crime] = pd.concat([dict_consolidado[crime], df], ignore_index=True)
        else:
            dict_consolidado[crime] = df
            
    return dict_consolidado



In [12]:
codigos = pd.read_csv("codigos_municipios_regioes.csv", sep=';',
                      encoding="ISO-8859-1")
codigos.drop(columns=['cod_ra', 'cod_ibge6', 'pop2020', 'porte_pop2020', 'cod_drs', 'drs'],
             axis=1, inplace=True)
codigos = codigos.iloc[1:, :]

In [15]:
dict_bases = {}
dict_concatenado = {}
leitura_bases('Dados_ssp_sp', dict_bases)
for ano, regioes in dict_bases.items():
    dict_concatenado[ano] = {}
    for regiao, municipios in regioes.items():
        for municipio, df in municipios.items():
            dict_bases[ano][regiao][municipio] = processar_base_municipio(df, municipio, codigos, ano)
            concatenar_municipios(dict_bases[ano][regiao][municipio], dict_concatenado[ano])